# General Features of the Type System



## Dynamic

Types are properties of values, not of variables

In [1]:
x = 1
println(typeof(x))
x = "asl;kdf"
println(typeof(x))

Int64
String


## Strong and Nominal

The subtyping relation is quite strict, and not based on structure, but on explicitely declared "inheritance". 

In [2]:
abstract A
type B <: A
    foo :: Int
end

type LikeB
    foo :: Int
end

In [3]:
LikeB <: B

false

In [4]:
isa(B(52), B)

true

In [5]:
isa(LikeB(52), B)

false

In [6]:
function printFoo(b::B)
    println("foo: $(b.foo)")
end

printFoo(B(52))
printFoo(LikeB(52))

foo: 52


LoadError: LoadError: MethodError: no method matching printFoo(::LikeB)
Closest candidates are:
  printFoo(!Matched::B) at In[6]:2
while loading In[6], in expression starting on line 6

In [7]:
function printAnyFoo(a_with_foo)
    println("foo: $(a_with_foo.foo), of type $(typeof(a_with_foo))")
end

printAnyFoo(B(52))
printAnyFoo(LikeB(52))

foo: 52, of type B
foo: 52, of type LikeB


There are abstract types, which are the only way to define a hierarchy:

In [8]:
Signed <: Integer <: Real <: Number

true

## Parametric

Types and methods can have type parameters, which can be constrained.

In [9]:
function dot{T <: Number}(v::Vector{T}, w::Vector{T})
    return sum(v .* w)
end

dot (generic function with 1 method)

In [10]:
dot([1//2], [1//4])

1//8

In [11]:
dot([1//2], [0.25])

LoadError: LoadError: MethodError: no method matching dot(::Array{Rational{Int64},1}, ::Array{Float64,1})
you may have intended to import Base.dot
Closest candidates are:
  dot{T<:Number}(::Array{T<:Number,1}, !Matched::Array{T<:Number,1}) at In[9]:2
while loading In[11], in expression starting on line 1

# Bits types

Are used to declare explicit storage size.

In [12]:
bitstype 8 ASCII

In [13]:
reinterpret(ASCII, UInt8(40))

ASCII(0x28)

# Type Hierarchy/Properties

In [14]:
Union{} <: Int32 <: Any

true

In [15]:
typealias Strange Union{Int, String}
Int <: Strange && String <: Strange

true

In [16]:
typejoin(Int, String)

Any

In [17]:
typejoin(UInt32, UInt16)

Unsigned

In [18]:
typeintersect(Signed, Unsigned)

Union{}

# Multimethods

"Methods" are associated with _functions_, not objects. This allows dynamic dispatch on the runtime type of multiple types (all arguments of a function), not just one special one (ie., `this`).

In [19]:
abstract Animal
immutable Tiger <: Animal end
immutable Lion <: Animal end

function breed(t1::Tiger, t2::Tiger)
    println("Bred new tiger!")
end

function breed(l1::Lion, l2::Lion)
    println("Bred new lion!")
end 

function breed(t::Tiger, l::Lion)
    println("Bred new tion!")
end

function breed(l::Lion, t::Tiger)
    println("Bred new liger!")
end 

a = Lion()
b = Tiger()

breed(a, b)

Bred new liger!


This is used a lot for specialization of code based on specific types, a very practical thing for efficient numerical computing.

In [20]:
methods(.*)

# 35 methods for generic function ".*":
.*(x::Bool, B::BitArray) at bitarray.jl:1250
.*(x::BitArray{1}, y::AbstractSparseArray{Bool,Ti<:Any,1}) at sparse/sparsevector.jl:1170
.*(x::Real, r::OrdinalRange) at range.jl:724
.*(x::Real, r::FloatRange) at range.jl:725
.*(x::Real, r::LinSpace) at range.jl:726
.*(r::FloatRange, x::Real) at range.jl:728
.*(r::LinSpace, x::Real) at range.jl:729
.*(r::Range, x::Real) at range.jl:727
.*(x::Number, r::Range) at range.jl:806
.*(r::Range, y::Number) at range.jl:807
.*(x::Number, y::Number) at operators.jl:150
.*(x::Number, B::BitArray) at bitarray.jl:1252
.*{P<:Base.Dates.Period}(y::Real, X::Union{Base.ReshapedArray{P,N<:Any,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray{P,N<:Any},SubArray{P,N<:Any,A<:Union{Base.ReshapedArray{T<:Any,N<:Any,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int64,Range{Int64}},N<:Any}},L<:Any}}) at dates/periods.jl:91
.*(A::Number, B::SparseMatrixCSC) at sparse/sparsematrix.jl:1720
.*(a::Number, x::AbstractSparseArray{Tv<:Any,Ti<:Any,1}) at sparse/sparsevector.jl:1288
.*{T}(A::Number, B::AbstractArray{T,N<:Any}) at arraymath.jl:67
.*(B::BitArray, x::Bool) at bitarray.jl:1251
.*(B::BitArray, x::Number) at bitarray.jl:1253
.*{P<:Base.Dates.Period}(X::Union{Base.ReshapedArray{P,N<:Any,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray{P,N<:Any},SubArray{P,N<:Any,A<:Union{Base.ReshapedArray{T<:Any,N<:Any,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int64,Range{Int64}},N<:Any}},L<:Any}}, y::Real) at dates/periods.jl:101
.*(A::SparseMatrixCSC, B::Number) at sparse/sparsematrix.jl:1719
.*(x::AbstractSparseArray{Tv<:Any,Ti<:Any,1}, a::Number) at sparse/sparsevector.jl:1287
.*{T}(A::AbstractArray{T,N<:Any}, B::Number) at arraymath.jl:77
.*{T<:AbstractString}(v::Array{T,1}, s::AbstractString) at strings/basic.jl:85
.*{T<:AbstractString}(s::AbstractString, v::Array{T,1}) at strings/basic.jl:86
.*(x::Number, J::UniformScaling) at linalg/uniformscaling.jl:152
.*(J::UniformScaling, x::Number) at linalg/uniformscaling.jl:153
.*(A::BitArray, B::BitArray) at broadcast.jl:432
.*(x::AbstractSparseArray{Bool,Ti<:Any,1}, y::BitArray{1}) at sparse/sparsevector.jl:1171
.*(A::AbstractArray{Bool,N<:Any}, B::BitArray) at broadcast.jl:432
.*(A::BitArray, B::AbstractArray{Bool,N<:Any}) at broadcast.jl:432
.*(x::AbstractSparseArray{Tv<:Any,Ti<:Any,1}, y::AbstractSparseArray{Tv<:Any,Ti<:Any,1}) at sparse/sparsevector.jl:1179
.*(x::Union{Base.ReshapedArray{T<:Any,1,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray{T<:Any,1},SubArray{T<:Any,1,A<:Union{Base.ReshapedArray{T<:Any,N<:Any,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int64,Range{Int64}},N<:Any}},L<:Any}}, y::AbstractSparseArray{Tv<:Any,Ti<:Any,1}) at sparse/sparsevector.jl:1180
.*(x::AbstractSparseArray{Tv<:Any,Ti<:Any,1}, y::Union{Base.ReshapedArray{T<:Any,1,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray{T<:Any,1},SubArray{T<:Any,1,A<:Union{Base.ReshapedArray{T<:Any,N<:Any,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int64,Range{Int64}},N<:Any}},L<:Any}}) at sparse/sparsevector.jl:1181
.*(A::AbstractArray, B::AbstractArray) at sparse/sparsematrix.jl:1718
.*(As::AbstractArray...) at broadcast.jl:305

Now, this would seem like we then have to implement quadratically many methods -- but no! There's a built-in mechanism of conversion and promotion, based on multimethods. Consider, for example, [these methods](http://docs.julialang.org/en/release-0.5/manual/conversion-and-promotion/#case-study-rational-conversions) for `Rational`:

In [21]:
convert{T<:Integer}(::Type{Rational{T}}, x::Rational) = Rational(convert(T,x.num),convert(T,x.den))
convert{T<:Integer}(::Type{Rational{T}}, x::Integer) = Rational(convert(T,x), convert(T,1))

function convert{T<:Integer}(::Type{Rational{T}}, x::AbstractFloat, tol::Real)
    if isnan(x); return zero(T)//zero(T); end
    if isinf(x); return sign(x)//zero(T); end
    y = x
    a = d = one(T)
    b = c = zero(T)
    while true
        f = convert(T,round(y)); y -= f
        a, b, c, d = f*a+c, f*b+d, a, b
        if y == 0 || abs(a/b-x) <= tol
            return a//b
        end
        y = 1/y
    end
end
convert{T<:Integer}(rt::Type{Rational{T}}, x::AbstractFloat) = convert(rt,x,eps(x))

convert{T<:AbstractFloat}(::Type{T}, x::Rational) = convert(T,x.num)/convert(T,x.den)
convert{T<:Integer}(::Type{T}, x::Rational) = div(convert(T,x.num),convert(T,x.den))

convert (generic function with 6 methods)

In [22]:
a = 1 + 2im
b = 3//4
println((typeof(a), typeof(b)))

c = promote(a, b)
println(c, typeof(c))

(Complex{Int64},Rational{Int64})
(1//1 + 2//1*im,3//4 + 0//1*im)Tuple{Complex{Rational{Int64}},Complex{Rational{Int64}}}


`promote` should not be overloaded directly -- instead, we can define _promote rules_ for tuples of types:

In [23]:
promote_rule(::Type{UInt8}, ::Type{Int8}) = Int
promote_rule(::Type{BigInt}, ::Type{Int8}) = BigInt

promote_rule (generic function with 2 methods)

# Things that I miss

## Function types

Unfortunately, function types are opaque -- there is only `Function`, which subsumes everthing. No `Function{X, Y}`.

In [24]:
f(x::Int) = 2x + 1
typeof(f)

#f

In [25]:
typeof(f) <: Function

true

## Co/Contravariance of Type Constructors

All types (except the built-in special case of `Tuple`) are invariant.

In [26]:
# we know: Int <: Number
Array{Int} <: Array{Number}

false

In [27]:
Tuple{Int, String} <: Tuple{Number, AbstractString}

true

In [28]:
function mysum(v::Array{Number})
    return sum(v)
end

println(mysum(Number[1,2,3]))
println(mysum([1,2,3]))

6


LoadError: LoadError: MethodError: no method matching mysum(::Array{Int64,1})
Closest candidates are:
  mysum(!Matched::Array{Number,N}) at In[28]:2
while loading In[28], in expression starting on line 6

In [29]:
# the usual workaround:
function mysum2{T <: Number}(v::Array{T})
    return sum(v)
end

mysum2 (generic function with 1 method)